### Тема: предсказание итоговой стоимости (Revised Cost)

In [55]:
import yaml
with open('../config.yaml', 'r') as config:
    cfg = yaml.safe_load(config)

In [56]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import json

In [57]:
df = pd.read_csv(cfg["dataset_1"])

/tmp/ipykernel_22562/2154587262.py:1: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cfg["dataset_1"])


In [58]:
df.head(10)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992
5,201706149344,8,otc alterations permit,06/14/2017,4105,009,800,NaN,Indiana,St,...,1.0,constr type 1,1.0,constr type 1,NaN,10.0,Potrero Hill,94107.0,"(37.75922331346539, -122.39170402628598)",1466911170855
6,201706300814,8,otc alterations permit,06/30/2017,1739,020,1291,NaN,11th,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,5.0,Inner Sunset,94122.0,"(37.764145640138565, -122.46875112470363)",1468970403692
7,M803667,8,otc alterations permit,06/30/2017,4789,014,1465,NaN,Revere,Av,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73005099023611, -122.38784938916618)",1469035175050
8,M804227,8,otc alterations permit,07/05/2017,1212,054,2094,NaN,Fell,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Lone Mountain/USF,94117.0,"(37.772393498502595, -122.45231466824669)",1469198253772
9,M804767,8,otc alterations permit,07/06/2017,1259,016,89,NaN,Alpine,Tr,...,NaN,NaN,NaN,NaN,NaN,8.0,Haight Ashbury,94117.0,"(37.7691724293766, -122.43734859051908)",146932394171


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197174 entries, 0 to 198885
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Permit Type                             197174 non-null  int64  
 1   Permit Type Definition                  197174 non-null  object 
 2   Permit Creation Date                    197174 non-null  object 
 3   Block                                   197174 non-null  object 
 4   Lot                                     197174 non-null  object 
 5   Street Number                           197174 non-null  int64  
 6   Street Name                             197174 non-null  object 
 7   Street Suffix                           197174 non-null  object 
 8   Unit                                    197174 non-null  float64
 9   Unit Suffix                             197174 non-null  object 
 10  Current Status                          197174 no

In [106]:
nan_df = (df.isnull().mean() * 100).reset_index()
nan_df.columns = ["column_name", "percentage"]
nan_df.sort_values("percentage", ascending=False, inplace=True)
nan_df.head(43)

,column_name,percentage
14,Completed Date,51.086857
21,Permit Expiration Date,25.977563
30,Existing Construction Type Description,21.678822
32,Proposed Construction Type Description,21.554566
26,Proposed Use,21.188899
24,Existing Use,20.540233
22,Estimated Cost,18.997941
15,First Construction Document Date,7.549170
13,Issued Date,7.546127
23,Revised Cost,3.069370


In [61]:
# Existing Construction Type, Proposed Construction Type, Number of Proposed Stories, Number of Existing Stories, Proposed Use, Existing Use
column = 'Neighborhoods - Analysis Boundaries'
this = set(df.dropna(subset=[column])[column])
len(this), this

(41,
 {'Bayview Hunters Point',
  'Bernal Heights',
  'Castro/Upper Market',
  'Chinatown',
  'Excelsior',
  'Financial District/South Beach',
  'Glen Park',
  'Golden Gate Park',
  'Haight Ashbury',
  'Hayes Valley',
  'Inner Richmond',
  'Inner Sunset',
  'Japantown',
  'Lakeshore',
  'Lincoln Park',
  'Lone Mountain/USF',
  'Marina',
  'McLaren Park',
  'Mission',
  'Mission Bay',
  'Nob Hill',
  'Noe Valley',
  'North Beach',
  'Oceanview/Merced/Ingleside',
  'Outer Mission',
  'Outer Richmond',
  'Pacific Heights',
  'Portola',
  'Potrero Hill',
  'Presidio',
  'Presidio Heights',
  'Russian Hill',
  'Seacliff',
  'South of Market',
  'Sunset/Parkside',
  'Tenderloin',
  'Treasure Island',
  'Twin Peaks',
  'Visitacion Valley',
  'West of Twin Peaks',
  'Western Addition'})

In [105]:
column = 'Supervisor District'
this = set(df[df[column].isna()]['Unit'])
len(this), this

(0, set())

**Выводы:**
 1. Количество столбцов с TIDF Compilance != NaN 2 и они принимают разные значения => это признак не поможет в обучении модели
 2. Voluntary Soft-Story Retrofit принимает в столбцах где не NaN значения Y, что значит True. NaN можно заменить на False и привести тип к булевому
 3. Unit Suffix можно объеденить в группы, значительно уменьшив уникальность, а пустые значения заменить на Not
 4. Street Suffix и Stree Number Suffix можно объеденить вместе
 5. Site Permit - наличие разрешения, принимает только одно значения Y, значит остальные можно заполнить N и привести к булевому типу. Аналогично с Structural Notification, Fire Only Permit
 6. Unit - номер квартиры, не может начинаться с 0, значит за 0 в датасете обозначается отсутствие у квартиры номера => заменим NaN на 0. Также NaN заменим 0 в атрибутах Existing Units, Proposed Units, Existing Construction Type, Proposed Construction Type, Number of Proposed Stories, Number of Existing Stories, Plansets
 7. Supervisor District - некоторые округа существуют без надзорщиков, поэтому будет считать, что если он не указан, то его нет и заменим NaN на 0
 8. Недостающие Zipcode и Neighborhoods - Analysis Boundaries определим при помощи location, которую разобьем на 2 признака: широту и долготу. Если она отсутствует - удаляем строку

### Заполнение отсутствующих значений

In [63]:
for column in ['Existing Construction Type',
               'Proposed Construction Type',
               'Number of Proposed Stories',
               'Number of Existing Stories',
               'Unit',
               'Existing Units',
               'Plansets',
               'Supervisor District',
               'Proposed Units']:
    df[column] = df[column].fillna(0)

In [64]:
for column in ['Site Permit', 'Voluntary Soft-Story Retrofit', 'Structural Notification', 'Fire Only Permit']:
    df[column] = df[column].fillna("N")
    df[column] = df[column].replace({"Y": 1, "N": 0})

/tmp/ipykernel_22562/2223798645.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace({"Y": 1, "N": 0})


In [65]:
df['Unit Suffix'] = df['Unit Suffix'].fillna("Not")

unit_suffix = None

with open('Unit-Suffix.json', 'r') as json_file:
    unit_suffix = json.load(json_file)
    
df['Unit Suffix'] = df['Unit Suffix'].replace(unit_suffix)

In [66]:
df['Street Suffix'] = df.apply(lambda row: f"{row['Street Suffix']}, {row['Street Number Suffix']}" if pd.notna(row['Street Number Suffix']) else row['Street Suffix'], axis=1)
df = df.drop(columns=['Street Number Suffix'])
df['Street Suffix'] = df['Street Suffix'].fillna("Not")

NameError: name 'fdgdgdfgd' is not defined

In [73]:
df = df.dropna(subset=['Location'])

df[['latitude', 'longitude']] = df['Location'].str.strip('()').str.split(',', expand=True)

df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [74]:
drop_columns = ['Location', 'Description', 'Permit Number', 'TIDF Compliance']
df = df.drop(columns=drop_columns)

In [92]:
df[['Neighborhoods - Analysis Boundaries', 'Zipcode']]

,Neighborhoods - Analysis Boundaries,Zipcode
0,Tenderloin,94102.0
1,Tenderloin,94102.0
2,Russian Hill,94109.0
3,Nob Hill,94109.0
4,Tenderloin,94102.0
...,...,...
198881,North Beach,94133.0
198882,Excelsior,94112.0
198883,Tenderloin,94102.0
198884,Mission,94103.0


In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
from geopy.exc import GeocoderTimedOut


geolocator = Nominatim(user_agent="Arnoochka")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

def get_location_info(row, max_retries=3, delay_seconds=1):
    latitude = row['latitude']
    longitude = row['longitude']
    
    retries = 0
    while retries < max_retries:
        try:
            location = reverse((latitude, longitude), exactly_one=True)
            address = location.raw['address']
            
            neighborhood = address.get('neighbourhood', None)
            postcode = address.get('postcode', None)
            
            return [neighborhood, postcode]
        
        except GeocoderTimedOut:
            retries += 1
            if retries < max_retries:
                time.sleep(delay_seconds)
            else:
                print(f"Timed out after {max_retries} attempts for coordinates ({latitude}, {longitude})")
                return [None, None]
        except Exception as e:
            print(f"Error occurred: {e} for coordinates ({latitude}, {longitude})")
            return [None, None]

for i in df.index:
    row = df.loc[i]
    if pd.isna(row['Neighborhoods - Analysis Boundaries']) or pd.isna(row['Zipcode']):
        neighborhood, zipcode = get_location_info(row)
        
        if pd.isna(row['Neighborhoods - Analysis Boundaries']): 
            df.loc[i]['Neighborhoods - Analysis Boundaries'] = neighborhood
            
        if pd.isna(row['Zipcode']):
           df.loc[i]['Zipcode'] = zipcode


df = df.dropna(subset=['Neighborhoods - Analysis Boundaries', 'Zipcode'])

/tmp/ipykernel_22562/2571415758.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i]['Neighborhoods - Analysis Boundaries'] = neighborhood
/tmp/ipykernel_22562/2571415758.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i]['Zipcode'] = zipcode
RateLimiter caught an error, retrying (0/2 tries). Called with (*((np.float64(37.708255336721685), np.float64(-122.44291501224802)),), **{'exactly_one': True}).
Traceback (most recent call last):
  File "/home/victor/anaconda3/envs/statistic/lib/python3.13/site-packages/geopy/adapters.py", line 298, in get_text
    page = self.urlopen(req, timeout=timeout)
  

### Работа с датой

### Разбиение признаков на категориальные и числовые

In [45]:
num_df = df.select_dtypes(include=['int64', 'float64'])
cat_df = df.select_dtypes(include=['object'])

### Работа с числовыми признаками

### Работа с категориальными данными